<a href="https://colab.research.google.com/github/KumudaBG/DeepLearning-Assignment1-fast.ai-tensorflow/blob/main/fast.ai/fastai_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**FAST.AI**
using the package fast.ai we are implementing the below learners
* cnn_learner
* unet_learner
* text_classifier_learner
* tabular_learner
* collab_learner

In [1]:
pip install fastai==2.2.5

In [2]:
from fastai.vision.all import *

download and decompress dataset from the collection fast.ai

In [3]:
pets_path = untar_data(URLs.PETS)/'images'


method returns true if the filename starts with lowercase

In [4]:
def is_dog(x): return x[0].islower()

#**cnn_learner**


* ImageDataLoaders - load the images
* item_tfms - predefined Transform to resize the item to 224 pixels
* valid_pct - validation set 20%, training set 80%
* label_func- check if the image is dog
* seed=42 set the random seed to a particular value/same validation set

In [5]:
dataloader = ImageDataLoaders.from_name_func(
    pets_path, get_image_files(pets_path), valid_pct=0.2, seed=42,
    label_func=is_dog, item_tfms=Resize(224))

* cnn- convolution neural network
* resnet- architecture with 34 layers
* metrics- method to measure the quality of prediction (error_rate or accuracy)

In [ ]:
learner = cnn_learner(dataloader, resnet34, metrics=error_rate)
#fit the model
learner.fine_tune(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.149843,0.032508,0.009472,00:46


epoch,train_loss,valid_loss,error_rate,time


Predict the learner

In [ ]:
from google.colab import files
#upload any type of file- download and upload dog image to predict the model 
uploaded = files.upload()

In [ ]:
image_uploader = SimpleNamespace(data = ['/content/dog.jpeg'])
image = PILImage.create(image_uploader.data[0])
is_dog,_,probabiltiy = learner.predict(image)
print(f"Is this a dog?: {is_dog}.")
print(f"Probability it's a dog: {probabiltiy[1].item():.6f}")

#**unet_learner**

In [ ]:
#download dataset from the collection fast.ai
camvid_dataset_path = untar_data(URLs.CAMVID_TINY)
segment_dataloader = SegmentationDataLoaders.from_label_func(
    camvid_dataset_path, bs=8, fnames = get_image_files(camvid_dataset_path/"images"),
    label_func = lambda o: camvid_dataset_path/'labels'/f'{o.stem}_P{o.suffix}',
    codes = np.loadtxt(camvid_dataset_path/'codes.txt', dtype=str)
)


In [ ]:
segmentation_learner = unet_learner(segment_dataloader, resnet34)
segmentation_learner.fine_tune(8)

In [ ]:
segmentation_learner.show_results(max_n=6, figsize=(7,8))

#**text_classifier_learner**

In [ ]:
from fastai.text.all import *

text_dataloader = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')
text_learner = text_classifier_learner(text_dataloader, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
text_learner.fine_tune(4, 1e-2)

In [ ]:
text_learner.predict("I love to to watch old Kannada movie!")

#**tabular_learner**

In [ ]:
from fastai.tabular.all import *
path = untar_data(URLs.ADULT_SAMPLE)

#cat_names-categorical columns
#cont_names-continous columns
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation',
                 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])
#predict a particular column of the row based on others column contents of the same row
# predict the label-salary
learn = tabular_learner(dls, metrics=accuracy)

In [ ]:
#fit_one_cycle- method to train fastai models from beginning
learn.fit_one_cycle(3)

#**collab_learner**

In [ ]:
#To generate a recommendation model from previous view habits
from fastai.collab import *
path = untar_data(URLs.ML_SAMPLE)
dls = CollabDataLoaders.from_csv(path/'ratings.csv')
learn = collab_learner(dls, y_range=(0.5,5.5))
learn.fine_tune(10)

In [ ]:
learn.show_results()